In [6]:

import pandas as pd
import sqlite3
from contextlib import closing
import psycopg2

def extract_aact():
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        
        cursor.execute(
            """
        SELECT ctgov_group_code, param_value
        FROM baseline_measurements
        WHERE title = 'Age, Continuous'
        AND nct_id = 'NCT01945775';
        """
            ),
        patients = cursor.fetchall()

    columns = [
        'group_code',
        'mean_age',
    ]

    df = pd.DataFrame(patients, columns=columns)

    print(df)



extract_aact()

    

  group_code mean_age
0      BG000     47.5
1      BG001     49.4
2      BG002     48.1
